<a href="https://colab.research.google.com/github/sweskr/Practice-works/blob/main/1_Month_Financial_Simulation_Report_(BTC_%26_Gold).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# --- 1. Import Libraries ---
import yfinance as yf
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import pandas_datareader.data as web
import matplotlib.pyplot as plt

# --- 2. Set Dates and Download Data ---
start = "2025-01-01"
end = "2025-09-30"

btc = yf.download("BTC-USD", start=start, end=end, interval="1mo")["Close"]
gold = yf.download("GC=F", start=start, end=end, interval="1mo")["Close"]
cpi = web.DataReader("CPIAUCSL", "fred", start, end)

# Combine into one DataFrame
df = pd.concat([btc, gold, cpi], axis=1)
df.columns = ["BTC", "Gold", "CPI"]

# Drop missing rows
df = df.dropna()

# --- 3. Compute Log Returns ---
returns = np.log(df / df.shift(1)).dropna()

# --- 4. Volatility (Annualized Std Dev) ---
vol = returns.std() * np.sqrt(12)

# --- 5. Correlation Matrix ---
corr = returns.corr()

# --- 6. Sharpe Ratios ---
risk_free_rate = 0.04
excess_returns = returns - (risk_free_rate / 12)
sharpe = (excess_returns.mean() * 12) / (returns.std() * np.sqrt(12))

# --- 7. Print Summary Statistics ---
print("Volatility (annualized):\n", vol, "\n")
print("Correlation matrix:\n", corr, "\n")
print("Sharpe Ratios:\n", sharpe, "\n")

# --- 8. Monte Carlo Simulation: Multivariate Normal Returns ---
mu = returns.mean().values
cov_matrix = returns.cov().values
n_simulations = 100000
n_assets = len(returns.columns)

simulated_returns = np.random.multivariate_normal(mu, cov_matrix, n_simulations)
simulated_df = pd.DataFrame(simulated_returns, columns=returns.columns)

expected_return = simulated_df.mean() * 12
simulated_vol = simulated_df.std() * np.sqrt(12)
sim_sharpe_ratio = expected_return / simulated_vol

print("Monte Carlo 1-Month Simulation Results (Annualized):\n")
print("Expected Returns:\n", expected_return)
print("\nVolatility:\n", simulated_vol)
print("\nSharpe Ratios:\n", sim_sharpe_ratio)

# --- 9. Monte Carlo: GBM Simulation for Price Forecast ---
np.random.seed(42)
n_sims = 5000
n_days = 30

# Initial Prices (from most recent data)
P0 = {
    "BTC": 116790.41,
    "Gold": 3706.00,
}

# Annualized Return Estimates (can adjust with real estimates)
mu = {
    "BTC": 0.113124,
    "Gold": 0.421625,
}

# Volatility Estimates
sigma = {
    "BTC": 0.65,
    "Gold": 0.15,
}

# GBM Simulation Function
def simulate_prices(asset, P0, mu, sigma, n_sims, n_days):
    dt = 1/252
    paths = np.zeros((n_days, n_sims))
    paths[0] = P0
    for t in range(1, n_days):
        rand = np.random.normal(0, 1, n_sims)
        paths[t] = paths[t-1] * np.exp((mu[asset] - 0.5*sigma[asset]**2)*dt + sigma[asset]*np.sqrt(dt)*rand)
    return paths

# Simulate BTC and Gold prices
btc_paths = simulate_prices("BTC", P0["BTC"], mu, sigma, n_sims, n_days)
gold_paths = simulate_prices("Gold", P0["Gold"], mu, sigma, n_sims, n_days)

# --- 10. Summary of Final Simulated Prices ---
final_prices = pd.DataFrame({
    "BTC Final Price": btc_paths[-1],
    "Gold Final Price": gold_paths[-1],
})

print("Expected Prices after 1 Month (Mean):")
print(final_prices.mean())

print("\n5% Worst Case (VaR-like):")
print(final_prices.quantile(0.05))

print("\n95% Best Case:")
print(final_prices.quantile(0.95))

# --- 11. Returns and Summary Statistics ---
returns_pct = (final_prices - pd.Series(P0)) / pd.Series(P0) * 100

summary = pd.DataFrame({
    "Mean Return %": returns_pct.mean(),
    "5% Worst Case %": returns_pct.quantile(0.05),
    "95% Best Case %": returns_pct.quantile(0.95),
    "Win Probability": (returns_pct > 0).mean() * 100
})

print("\n--- Summary of Simulated Returns (%) ---")
print(summary.round(2))


/tmp/ipython-input-378233589.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  btc = yf.download("BTC-USD", start=start, end=end, interval="1mo")["Close"]
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-378233589.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  gold = yf.download("GC=F", start=start, end=end, interval="1mo")["Close"]
[*********************100%***********************]  1 of 1 completed


Volatility (annualized):
 BTC     0.440022
Gold    0.138203
CPI     0.006710
dtype: float64 

Correlation matrix:
            BTC      Gold       CPI
BTC   1.000000 -0.086398  0.063322
Gold -0.086398  1.000000 -0.432258
CPI   0.063322 -0.432258  1.000000 

Sharpe Ratios:
 BTC     0.160832
Gold    2.766180
CPI    -1.991716
dtype: float64 

Monte Carlo 1-Month Simulation Results (Annualized):

Expected Returns:
 BTC     0.121500
Gold    0.419811
CPI     0.026559
dtype: float64

Volatility:
 BTC     0.439652
Gold    0.138977
CPI     0.006706
dtype: float64

Sharpe Ratios:
 BTC     0.276355
Gold    3.020719
CPI     3.960267
dtype: float64
Expected Prices after 1 Month (Mean):
BTC Final Price     118500.218116
Gold Final Price      3889.074602
dtype: float64

5% Worst Case (VaR-like):
BTC Final Price     80654.976809
Gold Final Price     3574.385944
Name: 0.05, dtype: float64

95% Best Case:
BTC Final Price     165565.697135
Gold Final Price      4218.471786
Name: 0.95, dtype: float64

--- 

**One-Month Monte Carlo Simulation Report: BTC vs Gold**

**Overview**
This report provides a Monte Carlo-based simulation of the one-month price outlook for Bitcoin (BTC) and Gold, using historical data from the past 30 trading days. The goal was to model the future price distribution using the Geometric Brownian Motion (GBM) framework, widely applied in financial forecasting.

**Methodology**
The simulation utilized daily closing prices for BTC and Gold obtained from Yahoo Finance, along with CPI data from FRED. The log returns of both assets were computed and used to derive the expected annualized returns and volatilities. The GBM model assumes that asset prices follow a stochastic exponential growth model characterized by a constant drift (mean return) and volatility.

For each asset, 5,000 simulation paths were generated over a 30-day forecast horizon, representing one trading month. The random shocks used in these simulations were drawn from a standard normal distribution, and drift/volatility parameters were calibrated from historical data.

**Results**

**Bitcoin (BTC):**
The expected one-month return was 1.46%, with a 5% worst-case outcome (Value at Risk) of -30.94%. The 95% best-case scenario suggested potential gains up to 41.76%. The probability of BTC delivering a positive return over the forecast period was estimated at 48.08%.

Gold:
Gold displayed a mean return of 4.94%, with a far more limited downside of -3.55% at the 5% tail and an upside potential of 13.83%. The probability of a positive return was 82.68%, indicating a strong risk-adjusted outlook.

**Analysis**
The simulation highlights the classic behavior of these two assets: Bitcoin remains a highly volatile, speculative instrument with extreme return distribution, while Gold shows stability and consistent performance. Bitcoin’s broad return distribution is reflected in both its upside and downside volatility, with a Sharpe ratio below 0.3. Gold, on the other hand, exhibits a high Sharpe ratio (>2.7), driven by its steady returns and low volatility.

The win probability metric—indicating how often returns were positive across simulations—further underscores Gold’s dominance as a short-term safe haven. BTC’s 48% win rate resembles a coin flip, while Gold’s 82% probability suggests strong predictability and reliability in the short term.

**Conclusion**
This Monte Carlo simulation indicates that over the next month, investors seeking higher certainty and capital preservation may benefit from Gold exposure. Bitcoin continues to offer higher risk/reward dynamics but comes with elevated downside risk. These results are consistent with traditional asset allocation insights and reinforce the importance of balancing riskier assets with stable stores of value like Gold.